# **BirdCLEF 2025 Inference Notebook**
This notebook runs inference on BirdCLEF 2025 test soundscapes and generates a submission file. It supports both single model inference and ensemble inference with multiple models. You can find the pre-processing and training processes in the following notebooks:

- [Transforming Audio-to-Mel Spec. | BirdCLEF'25](https://www.kaggle.com/code/kadircandrisolu/transforming-audio-to-mel-spec-birdclef-25)  
- [EfficientNet B0 Pytorch [Train] | BirdCLEF'25](https://www.kaggle.com/code/kadircandrisolu/efficientnet-b0-pytorch-train-birdclef-25)

**Features**
- Audio Preprocessing
- Test-Time Augmentation (TTA)

In [ ]:
import os
import gc
import warnings
import logging
import time
import math
import cv2
from pathlib import Path

import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
import torchaudio
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)

In [ ]:
def set_seed(seed=42):
    """Set seeds for reproducibility."""
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
class CFG:
    seed = 42
 
    test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
    submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
    taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'
    model_path = '/kaggle/input/birdclef25-efficientnet-pseudlabled/pytorch/primary-only-thr-0.9/3'  
    
    # Audio parameters
    FS = 32000  
    WINDOW_SIZE = 5  
    TARGET_DURATION = 5.0
    
    # Mel spectrogram parameters
    N_FFT = 1024
    HOP_LENGTH = 512
    N_MELS = 128
    FMIN = 50
    FMAX = 14000
    TARGET_SHAPE = (256, 256)
    normal = 255
    
    # Model architecture - must match training notebook
    model_name = 'seresnext26t_32x4d'  # Match training notebook default
    in_channels = 1
    device = 'cpu'
    pretrained = False
    dropout_rate = 0.2
    drop_path_rate = 0.2
    
    # Inference parameters
    batch_size = 16
    use_tta = False  
    tta_count = 3
    threshold = 0.5
    
    use_specific_folds = False  # If False, use all found models
    folds = [0, 1]  # Used only if use_specific_folds is True
    
    debug = False
    debug_count = 3
    
    # Debug option for state dict loading
    debug_state_dict = False  # Set to True to print missing keys

cfg = CFG()
set_seed(cfg.seed)

In [67]:
print(f"Using device: {cfg.device}")
print(f"Loading taxonomy data...")
taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
species_ids = taxonomy_df['primary_label'].tolist()
num_classes = len(species_ids)
print(f"Number of classes: {num_classes}")

Using device: cpu
Loading taxonomy data...
Number of classes: 206


In [ ]:
class AttBlockV2(nn.Module):
    def __init__(self, in_features: int, out_features: int, activation="linear"):
        super().__init__()

        self.activation = activation
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True,
        )
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True,
        )

        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == "linear":
            return x
        elif self.activation == "sigmoid":
            return torch.sigmoid(x)


def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.0)

def init_bn(bn):
    bn.bias.data.fill_(0.0)
    bn.weight.data.fill_(1.0)


class BirdCLEFModel(nn.Module):
    def __init__(self, cfg, num_classes):
        super().__init__()
        self.cfg = {attr: getattr(cfg, attr) for attr in dir(cfg) if not attr.startswith('__') and not callable(getattr(cfg, attr))}
        self.num_classes = num_classes
        
        # Support for different model architectures - match training notebook
        self.backbone = timm.create_model(
            cfg.model_name,
            pretrained=cfg.pretrained,
            in_chans=cfg.in_channels,
            drop_rate=cfg.dropout_rate,
            drop_path_rate=cfg.drop_path_rate
        )
        self.bn0 = nn.BatchNorm2d(self.cfg['TARGET_SHAPE'][0])
        layers = list(self.backbone.children())[:-2]
        self.encoder = nn.Sequential(*layers)

        # Extract feature dimension based on model type
        if 'efficientnet' in cfg.model_name:
            backbone_out = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
        elif 'convnext' in cfg.model_name:
            backbone_out = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
        elif 'res' in cfg.model_name:
            backbone_out = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
        else:
            backbone_out = self.backbone.get_classifier().in_features
            self.backbone.reset_classifier(0, '')
        
        self.fc1 = nn.Linear(backbone_out, backbone_out, bias=True)
        # Change activation to linear to get raw logits
        self.att_block = AttBlockV2(backbone_out, self.num_classes, activation="linear")

        # IMPORTANT: These internal mel spectrogram transforms exist in the model but are NOT used
        # during inference. Instead, we use the external preprocessing from transforming-training-audio
        # notebook to ensure consistency with training data preprocessing.
        self.melspec_transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=self.cfg['FS'],
            hop_length=self.cfg['HOP_LENGTH'],
            n_mels=self.cfg['N_MELS'],
            f_min=self.cfg['FMIN'],
            f_max=self.cfg['FMAX'],
            n_fft=self.cfg['N_FFT'],
            pad_mode="constant",
            norm="slaney",
            onesided=True,
            mel_scale="htk",
        )
        if self.cfg['device'] == "cuda":
            self.melspec_transform = self.melspec_transform.cuda()
        else:
            self.melspec_transform = self.melspec_transform.cpu()

        self.db_transform = torchaudio.transforms.AmplitudeToDB(
            stype="power", top_db=80
        )

    def extract_feature(self,x):
        x = x.permute((0, 1, 3, 2))
        frames_num = x.shape[2]
        
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        
        x = x.transpose(2, 3)
        # (batch_size, channels, freq, frames)
        x = self.encoder(x)
        
        # (batch_size, channels, frames)
        x = torch.mean(x, dim=2)
        
        # channel smoothing
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2
        
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)
        return x, frames_num
        
    @torch.cuda.amp.autocast(enabled=False)
    def transform_to_spec(self, audio):
        audio = audio.float()
        
        spec = self.melspec_transform(audio)
        spec = self.db_transform(spec)

        if self.cfg['normal'] == 80:
            spec = (spec + 80) / 80
        elif self.cfg['normal'] == 255:
            spec = spec / 255
        else:
            raise NotImplementedError
        
        return spec

    def forward(self, x):
        # x is a mel spectrogram that was preprocessed externally using the same approach
        # as the transforming-training-audio notebook (NOT using self.transform_to_spec)
        x, frames_num = self.extract_feature(x)
        
        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        segmentwise_logit = self.att_block.cla(x).transpose(1, 2)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        # Return raw logits (no sigmoid/logit conversion)
        return clipwise_output

In [ ]:
def audio2melspec(audio_data, cfg):
    """
    Convert audio data to mel spectrogram using the EXACT same approach as the 
    transforming-training-audio notebook. This ensures consistency between 
    training and inference preprocessing.
    """
    if np.isnan(audio_data).any():
        mean_signal = np.nanmean(audio_data)
        audio_data = np.nan_to_num(audio_data, nan=mean_signal)

    mel_spec = librosa.feature.melspectrogram(
        y=audio_data,
        sr=cfg.FS,
        n_fft=cfg.N_FFT,
        hop_length=cfg.HOP_LENGTH,
        n_mels=cfg.N_MELS,
        fmin=cfg.FMIN,
        fmax=cfg.FMAX,
        power=2.0
    )

    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    
    # Use the EXACT normalization from transforming-training-audio notebook
    mel_spec_norm = (mel_spec_db - mel_spec_db.min()) / (mel_spec_db.max() - mel_spec_db.min() + 1e-8)
    
    return mel_spec_norm

def process_audio_segment(audio_data, cfg):
    """
    Process audio segment to get mel spectrogram using the EXACT same approach as the 
    transforming-training-audio notebook. This function converts raw audio to the 
    mel spectrogram format that the model expects.
    """
    if len(audio_data) < cfg.FS * cfg.WINDOW_SIZE:
        audio_data = np.pad(audio_data, 
                          (0, cfg.FS * cfg.WINDOW_SIZE - len(audio_data)), 
                          mode='constant')
    
    mel_spec = audio2melspec(audio_data, cfg)
    
    # Resize if needed
    if mel_spec.shape != cfg.TARGET_SHAPE:
        mel_spec = cv2.resize(mel_spec, cfg.TARGET_SHAPE, interpolation=cv2.INTER_LINEAR)
        
    return mel_spec.astype(np.float32)

In [ ]:
def find_model_files(cfg):
    """
    Find all .pth model files in the specified model directory
    """
    model_files = []
    
    model_dir = Path(cfg.model_path)
    
    for path in model_dir.glob('**/*.pth'):
        model_files.append(str(path))
    
    return model_files

def load_models(cfg, num_classes):
    """
    Load all found model files and prepare them for ensemble
    """
    models = []
    
    model_files = find_model_files(cfg)
    
    if not model_files:
        print(f"Warning: No model files found under {cfg.model_path}!")
        return models
    
    print(f"Found a total of {len(model_files)} model files.")
    
    if cfg.use_specific_folds:
        filtered_files = []
        for fold in cfg.folds:
            fold_files = [f for f in model_files if f"fold{fold}" in f]
            filtered_files.extend(fold_files)
        model_files = filtered_files
        print(f"Using {len(model_files)} model files for the specified folds ({cfg.folds}).")
    
    for model_path in model_files:
        try:
            print(f"Loading model: {model_path}")
            checkpoint = torch.load(model_path, map_location=cfg.device)
            
            model = BirdCLEFModel(cfg, num_classes)
            
            # Handle different state dict configurations
            state_dict = None
            if 'model_state_dict' in checkpoint:
                state_dict = checkpoint['model_state_dict']
            elif 'state_dict' in checkpoint:
                state_dict = checkpoint['state_dict']
            else:
                # Assume the checkpoint itself is the state dict
                state_dict = checkpoint
            
            # Handle DataParallel wrapped state dict (remove 'module.' prefix)
            if any(k.startswith('module.') for k in state_dict.keys()):
                state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
            
            # Check for missing keys before loading
            model_dict = model.state_dict()
            missing_keys = [k for k in model_dict.keys() if k not in state_dict]
            unexpected_keys = [k for k in state_dict.keys() if k not in model_dict]
            
            if missing_keys and cfg.debug_state_dict:
                print(f"Missing keys: {missing_keys}")
            if unexpected_keys and cfg.debug_state_dict:
                print(f"Unexpected keys: {unexpected_keys}")
            
            # Try to partially load if there are missing or unexpected keys
            if missing_keys or unexpected_keys:
                print(f"Warning: {len(missing_keys)} missing keys, {len(unexpected_keys)} unexpected keys")
                # Filter state_dict to only include keys that are in the model
                filtered_state_dict = {k: v for k, v in state_dict.items() if k in model_dict}
                model_dict.update(filtered_state_dict)
                model.load_state_dict(model_dict, strict=False)
                print("Loaded model with partial state dict")
            else:
                model.load_state_dict(state_dict)
                
            model = model.to(cfg.device)
            model.eval()
            
            models.append(model)
        except Exception as e:
            print(f"Error loading model {model_path}: {e}")
            import traceback
            traceback.print_exc()
    
    return models

def predict_on_spectrogram(audio_path, models, cfg, species_ids):
    """
    Process a single audio file and predict species presence for each 5-second segment.
    
    This function:
    1. Loads raw audio from the file
    2. Converts audio to mel spectrograms using the transforming-training-audio approach
    3. Feeds the spectrograms to the model for prediction
    """
    predictions = []
    row_ids = []
    soundscape_id = Path(audio_path).stem
    
    try:
        print(f"Processing {soundscape_id}")
        # Load raw audio
        audio_data, _ = librosa.load(audio_path, sr=cfg.FS)
        
        total_segments = int(len(audio_data) / (cfg.FS * cfg.WINDOW_SIZE))
        
        for segment_idx in range(total_segments):
            start_sample = segment_idx * cfg.FS * cfg.WINDOW_SIZE
            end_sample = start_sample + cfg.FS * cfg.WINDOW_SIZE
            segment_audio = audio_data[start_sample:end_sample]
            
            end_time_sec = (segment_idx + 1) * cfg.WINDOW_SIZE
            row_id = f"{soundscape_id}_{end_time_sec}"
            row_ids.append(row_id)

            if cfg.use_tta:
                all_preds = []
                
                for tta_idx in range(cfg.tta_count):
                    # Convert audio segment to mel spectrogram using transforming-training-audio approach
                    mel_spec = process_audio_segment(segment_audio, cfg)
                    mel_spec = apply_tta(mel_spec, tta_idx)

                    mel_spec = torch.tensor(mel_spec, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
                    mel_spec = mel_spec.to(cfg.device)

                    if len(models) == 1:
                        with torch.no_grad():
                            outputs = models[0](mel_spec)
                            probs = torch.sigmoid(outputs).cpu().numpy().squeeze()
                            all_preds.append(probs)
                    else:
                        segment_preds = []
                        for model in models:
                            with torch.no_grad():
                                outputs = model(mel_spec)
                                probs = torch.sigmoid(outputs).cpu().numpy().squeeze()
                                segment_preds.append(probs)
                        
                        avg_preds = np.mean(segment_preds, axis=0)
                        all_preds.append(avg_preds)

                final_preds = np.mean(all_preds, axis=0)
            else:
                # Convert audio segment to mel spectrogram using transforming-training-audio approach
                mel_spec = process_audio_segment(segment_audio, cfg)
                
                mel_spec = torch.tensor(mel_spec, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
                mel_spec = mel_spec.to(cfg.device)
                
                if len(models) == 1:
                    with torch.no_grad():
                        outputs = models[0](mel_spec)
                        final_preds = torch.sigmoid(outputs).cpu().numpy().squeeze()
                else:
                    segment_preds = []
                    for model in models:
                        with torch.no_grad():
                            outputs = model(mel_spec)
                            probs = torch.sigmoid(outputs).cpu().numpy().squeeze()
                            segment_preds.append(probs)

                    final_preds = np.mean(segment_preds, axis=0)
                    
            predictions.append(final_preds)
            
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
    
    return row_ids, predictions

In [ ]:
def apply_tta(spec, tta_idx):
    """Apply test-time augmentation"""
    if tta_idx == 0:
        # Original spectrogram
        return spec
    elif tta_idx == 1:
        # Time shift (horizontal flip)
        return np.flip(spec, axis=1)
    elif tta_idx == 2:
        # Frequency shift (vertical flip)
        return np.flip(spec, axis=0)
    else:
        return spec

def run_inference(cfg, models, species_ids):
    """Run inference on all test soundscapes"""
    test_files = list(Path(cfg.test_soundscapes).glob('*.ogg'))
    
    if cfg.debug:
        print(f"Debug mode enabled, using only {cfg.debug_count} files")
        test_files = test_files[:cfg.debug_count]
    
    print(f"Found {len(test_files)} test soundscapes")

    all_row_ids = []
    all_predictions = []

    for audio_path in tqdm(test_files):
        row_ids, predictions = predict_on_spectrogram(str(audio_path), models, cfg, species_ids)
        all_row_ids.extend(row_ids)
        all_predictions.extend(predictions)
    
    return all_row_ids, all_predictions

def create_submission(row_ids, predictions, species_ids, cfg):
    """Create submission dataframe"""
    print("Creating submission dataframe...")

    submission_dict = {'row_id': row_ids}
    
    for i, species in enumerate(species_ids):
        submission_dict[species] = [pred[i] for pred in predictions]

    submission_df = pd.DataFrame(submission_dict)

    submission_df.set_index('row_id', inplace=True)

    sample_sub = pd.read_csv(cfg.submission_csv, index_col='row_id')

    missing_cols = set(sample_sub.columns) - set(submission_df.columns)
    if missing_cols:
        print(f"Warning: Missing {len(missing_cols)} species columns in submission")
        for col in missing_cols:
            submission_df[col] = 0.0

    submission_df = submission_df[sample_sub.columns]

    submission_df = submission_df.reset_index()
    
    return submission_df

In [ ]:
def main():
    start_time = time.time()
    print("Starting BirdCLEF-2025 inference...")
    print(f"TTA enabled: {cfg.use_tta} (variations: {cfg.tta_count if cfg.use_tta else 0})")
    print(f"Using model architecture: {cfg.model_name}")

    models = load_models(cfg, num_classes)
    
    if not models:
        print("No models found! Please check model paths.")
        return
    
    print(f"Model usage: {'Single model' if len(models) == 1 else f'Ensemble of {len(models)} models'}")

    row_ids, predictions = run_inference(cfg, models, species_ids)

    submission_df = create_submission(row_ids, predictions, species_ids, cfg)

    submission_path = 'submission.csv'
    submission_df.to_csv(submission_path, index=False)
    print(f"Submission saved to {submission_path}")
    
    end_time = time.time()
    print(f"Inference completed in {(end_time - start_time)/60:.2f} minutes")

In [ ]:
def debug_model_compatibility():
    """
    Function to debug model compatibility issues
    """
    print("\n=== Debugging Model Compatibility ===")
    
    print(f"Creating a test model with the current configuration...")
    test_model = BirdCLEFModel(cfg, num_classes)
    test_state_dict = test_model.state_dict()
    
    print(f"Model has {len(test_state_dict)} parameters")
    
    # Display some parameter shapes to help debug
    print("\nSample parameter shapes from current model:")
    for i, (name, param) in enumerate(test_state_dict.items()):
        if i < 5 or i > len(test_state_dict) - 5:  # First 5 and last 5 params
            print(f"{name}: {param.shape}")
        if i == 5 and len(test_state_dict) > 10:
            print("...")
            
    # Try to load sample model
    model_files = find_model_files(cfg)
    if model_files:
        sample_file = model_files[0]
        print(f"\nAttempting to load a sample model: {sample_file}")
        
        try:
            checkpoint = torch.load(sample_file, map_location=cfg.device)
            
            # Check what's in the checkpoint
            if isinstance(checkpoint, dict):
                print(f"Checkpoint keys: {list(checkpoint.keys())}")
                
                # Look at state dict keys
                if 'model_state_dict' in checkpoint:
                    state_dict = checkpoint['model_state_dict']
                elif 'state_dict' in checkpoint:
                    state_dict = checkpoint['state_dict']
                else:
                    state_dict = None
                    print("No state dict found in checkpoint")
                
                if state_dict:
                    print(f"State dict has {len(state_dict)} parameters")
                    
                    # Count mismatches
                    missing = [k for k in test_state_dict.keys() if k not in state_dict]
                    unexpected = [k for k in state_dict.keys() if k not in test_state_dict]
                    
                    print(f"Missing keys: {len(missing)}")
                    print(f"Unexpected keys: {len(unexpected)}")
                    
                    if missing:
                        print("\nSample missing keys:")
                        for k in missing[:5]:
                            print(f"  {k}")
                    
                    if unexpected:
                        print("\nSample unexpected keys:")
                        for k in unexpected[:5]:
                            print(f"  {k}")
                        
                    # Try to map keys
                    if unexpected and missing:
                        print("\nAttempting to map keys...")
                        successful_maps = 0
                        for unexp_key in unexpected[:10]:
                            for miss_key in missing:
                                # Simple heuristic: check if the key ends with the same parameter name
                                if unexp_key.split('.')[-1] == miss_key.split('.')[-1]:
                                    print(f"Possible mapping: {unexp_key} -> {miss_key}")
                                    successful_maps += 1
                                    break
                        
                        if successful_maps == 0:
                            print("No obvious mappings found. Model architectures may be different.")
            else:
                print("Checkpoint is not a dictionary - it may be a direct state dict")
                
        except Exception as e:
            print(f"Error loading sample model: {e}")
            import traceback
            traceback.print_exc()
    
    print("\n=== End of Compatibility Debugging ===")

In [ ]:
if __name__ == "__main__":
    # Debug model compatibility if needed
    if cfg.debug_state_dict:
        debug_model_compatibility()
    
    main()

Starting BirdCLEF-2025 inference...
TTA enabled: False (variations: 0)
Found a total of 1 model files.
Loading model: /kaggle/input/birdclef25-efficientnet-pseudlabled/pytorch/primary-only-thr-0.9/3/model_20250501_131702_efficientnet_b0_fold0.pth
Error loading model /kaggle/input/birdclef25-efficientnet-pseudlabled/pytorch/primary-only-thr-0.9/3/model_20250501_131702_efficientnet_b0_fold0.pth: BirdCLEFModel.__init__() takes 2 positional arguments but 3 were given
No models found! Please check model paths.
